In [1]:
# (1) Dates and times in general
# (2) Dates and times in Pandas -- loading data
# (3) Intervals / timedeltas -- creating and calculating
# (4) Resampling and aggregate measures

In [2]:
# http://files.lerner.co.il/data-science-exercise-files.zip

# Two types of data structures for dates and times

- timestamp, datetime object: reflects a specific point in time
    - your birthday
    - your anniversary
    - when a meeting is going to happen
- timedelta: reflects a span of time — measured in years/months/etc., but no fixed point
    - how long is a meeting going to last?
    - how many seconds have you been alive?
    - how much longer must I sit through this class